<a href="https://colab.research.google.com/github/zloelias/TextAttack/blob/master/finetune_kinopoisk_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
! pip install datasets

In [2]:
! git config --global credential.helper store

In [3]:
! huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [4]:
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.0.2).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
Git LFS initialized.


In [5]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer



In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

cuda:0


In [7]:
from datasets import load_dataset

dataset = load_dataset('zloelias/kinopoisk-reviews')

Using custom data configuration zloelias___kinopoisk-reviews-2105e1c5dfdebfd7


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/18.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/162M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/zloelias___kinopoisk-reviews-2105e1c5dfdebfd7/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def count_classes(dataset, label_key='labels'):
    classes = set()
    dataset['train'].map(lambda x: classes.add(x[label_key]))
    return len(classes)

In [9]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

BERT_MODEL = "cointegrated/rubert-tiny2"

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, padding=True, truncation=True)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels = count_classes(dataset))
model.to(device)

  0%|          | 0/73820 [00:00<?, ?ex/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elemen

In [10]:
from transformers import TrainingArguments, Trainer

batch_size = 32

def preprocess_function(examples, sentence_key='text'):
    return tokenizer(examples[sentence_key], padding=True, truncation=True, max_length=512)

dataset = dataset.map(preprocess_function, batched=True)

#dataset['train'] = dataset['train'].select(list(range(0,1800))).map(preprocess_function, batched=True)
#dataset['test'] = dataset['test'].select(list(range(0,300))).map(preprocess_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/74 [00:00<?, ?ba/s]

In [11]:
metric_name = "accuracy"
dataset_name = "kinopoisk-reviews"
task = "clf"
model_name = BERT_MODEL.split("/")[-1]

batch_size = 64

args = TrainingArguments(
    f"{model_name}-{dataset_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=50,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

from sklearn.metrics import accuracy_score#, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': acc,
        #'precision': precision,
        #'recall': recall,
        #'f1': f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/rubert-tiny2-kinopoisk-reviews-finetuned-clf is already a clone of https://huggingface.co/zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf. Make sure you pull the latest changes with `repo.git_pull()`.


In [12]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: label_name, text, __index_level_0__.
***** Running training *****
  Num examples = 73820
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 3200
  Gradient Accumulation steps = 50
  Total optimization steps = 115


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.515337,0.743021
1,No log,0.289327,0.873705
2,No log,0.206801,0.920761
3,No log,0.167869,0.943679
4,No log,0.185017,0.948312


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: label_name, text, __index_level_0__.
***** Running Evaluation *****
  Num examples = 8203
  Batch size = 64
Saving model checkpoint to rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-23
Configuration saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-23/config.json
Model weights saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-23/pytorch_model.bin
tokenizer config file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-23/tokenizer_config.json
Special tokens file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-23/special_tokens_map.json
tokenizer config file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/tokenizer_config.json
Special tokens file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/special_tokens_map.json
The following columns in the evaluation set  

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.515337,0.743021
1,No log,0.289327,0.873705
2,No log,0.206801,0.920761
3,No log,0.167869,0.943679
4,No log,0.185017,0.948312


TrainOutput(global_step=115, training_loss=0.2928155650263247, metrics={'train_runtime': 2122.9372, 'train_samples_per_second': 173.863, 'train_steps_per_second': 0.054, 'total_flos': 2720197469306880.0, 'train_loss': 0.2928155650263247, 'epoch': 5.0})